# Processing SAR data to TileDB arrays

Data is openly licensed from the Spacenet 6 challenge -https://medium.com/the-downlinq/spacenet-6-expanded-dataset-release-e1a7ddaf030

We will start by copying over two sample SAR intensity GeoTIFFs.

In [5]:
import os

image_1 = "SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111224_20190804111453.tif"
image_2 = "SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111851_20190804112030.tif"

if not os.path.exists(f"./tmp/{image_1}"):
    !aws s3 cp s3://spacenet-dataset/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/$image_1 ./tmp/
    !aws s3 cp s3://spacenet-dataset/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/$image_2 ./tmp/

download failed: s3://spacenet-dataset/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111224_20190804111453.tif to tmp/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111224_20190804111453.tif Could not connect to the endpoint URL: "https://spacenet-dataset.s3.amazonaws.com/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111224_20190804111453.tif"
download failed: s3://spacenet-dataset/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111851_20190804112030.tif to tmp/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111851_20190804112030.tif Could not connect to the endpoint URL: "https://spacenet-dataset.s3.amazonaws.com/AOIs/AOI_11_Rotterdam/SAR-MAG-POL/SN6_AOI_11_Rotterdam_SAR-MAG-POL_20190804111851_20190804112030.tif"


In [ ]:
import json
from matplotlib import pyplot
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.warp import transform_bounds
from rasterio.windows import Window
from rasterio.crs import CRS
import shutil

output_array = "arrays/sar_output_array"

# clean up previous runs
try:
    shutil.rmtree(output_array)
except:
    pass

In [ ]:
import json
from matplotlib import pyplot
import numpy as np
import rasterio
from rasterio.plot import show
from rasterio.warp import transform_bounds
from rasterio.windows import Window
from rasterio.crs import CRS
import shutil

output_array = "output_array"

# clean up previous runs
try:
    shutil.rmtree(output_array)
except:
    pass

In [ ]:
!rio info $image_1 | jq .
!rio info $image_2 | jq .

We will mosaic the input GeoTIFF images

In [ ]:
!gdalbuildvrt mosaic.vrt SN6*.tif
!gdal_translate -OF TileDB -CO BLOCKXSIZE=1024 -CO BLOCKYSIZE=1024 -CO COMPRESSION=ZSTD -CO COMPRESSION_LEVEL=7 mosaic.vrt $output_array

In [ ]:
!rio info $output_array | jq .

Finally we will slice an AOI from the TileDB array using Rasterio and display

In [ ]:
from rasterio.plot import reshape_as_raster, reshape_as_image, adjust_band

with rasterio.Env():
    with rasterio.open(output_array) as src:
        # query dataset in lat/lon coordinates
        bbox = [4.42, 51.885, 4.435, 51.89]
        native_bounds = transform_bounds(CRS.from_epsg(4326), src.crs, *bbox)        
        bounds_window = src.window(*native_bounds)
        bounds_window = bounds_window.intersection(Window(0, 0, src.width, src.height))

        # The georeferenced MAG-POL data has a base resolution of 0.25m and contains 4 channels of SAR magnitude (intensity)
        # (1: HH , 2: HV, 3: VH, 4: VV) as well as 2 channels (5: alpha² & 6: beta²)
        # (Red = HH Intensity, Green = VV Intensity, Blue = HV Intensity)
        img = src.read((1, 4, 2), window=bounds_window)
        fig, ax = pyplot.subplots(1, 1, figsize=(21,7))
        ax.set_xlabel('X')
        ax.set_ylabel('Y')
        
        arr = reshape_as_raster(img)
        for b, band in enumerate(arr):
            arr[b] = adjust_band(band)
        arr = reshape_as_image(arr)        
        
        show(arr, ax=ax, title="Slicing SAR Intensity data from a TileDB array")